In [2]:
# import dependencies
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras import layers, Model
from src.utils import *
from src.gcn_model import *


In [3]:
# Load the dataset
cites_df = pd.read_csv('./cora/cora.cites', sep='\t',header=None, names=["start", "end"]) 
content_df = pd.read_csv('./cora/cora.content', sep='\t', header=None)
# The first column is the paper ID, the last is the class label, the rest are the features.

In [4]:
graph = nx.DiGraph()

# Add nodes with features
for _, row in content_df.iterrows():
    node_id = row[0]
    features = row[1:-1].values
    label = row.values[-1]
    graph.add_node(node_id, feature=features, label=label)

# Add edges from citations
for _, row in cites_df.iterrows():
    src = row['start']
    dst = row['end']
    graph.add_edge(src, dst)

In [15]:
labels= get_node_labels(graph)
labels

array(['Neural_Networks', 'Rule_Learning', 'Reinforcement_Learning', ...,
       'Genetic_Algorithms', 'Case_Based', 'Neural_Networks'],
      dtype='<U22')

In [20]:
def encode_label(labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = tf.keras.utils.to_categorical(labels)
    return labels, label_encoder.classes_

In [21]:
encode_label(labels)

(array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]]),
 array(['Case_Based', 'Genetic_Algorithms', 'Neural_Networks',
        'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning',
        'Theory'], dtype='<U22'))

In [16]:
len(labels)

2708

In [22]:
A= get_adjacency_matrix(graph)
A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# adjacency matrix
adj_matrix = nx.to_numpy_array(graph, nodelist=np.array(list(graph.nodes())))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
label_map, y, y_onehot, y_encoded = get_encoded_labels(graph)

In [10]:
adjacency_matrix = get_adjacency_matrix(graph)
A = get_symmetric_normalized_adjacency(adjacency_matrix)

In [55]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(content_df.iloc[:, -1])
onehot_encoder = OneHotEncoder(sparse_output=False)
y_onehot = onehot_encoder.fit_transform(y_encoded.reshape(-1, 1))
num_classes = y_onehot.shape[1]

In [73]:
paper_ids = content_df.iloc[:, 0].values
paper_ids

array([  31336, 1061127, 1106406, ..., 1128978,  117328,   24043])

In [83]:
np.unique(content_df.iloc[:, -1].values)

array(['Case_Based', 'Genetic_Algorithms', 'Neural_Networks',
       'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning',
       'Theory'], dtype=object)

In [ ]:

subjects = np.unique(content_df.iloc[:, -1].values)
num_nodes = len(paper_ids)

In [ ]:
# Create the directed graph from the cites data
G = nx.from_pandas_edgelist(cites_df, 'start', 'end', create_using=nx.DiGraph())

# Modelling

## GCN

3. Develop a machine learning approach to predict the subjects of papers.
4. Train your approach on 9 folds, evaluate it on 1 fold, repeat this process 10 times, and concatenate your predictions such that you have a prediction for every data point in the end.
5. Store your predictions in a file as tab-separated values (TSV) in the format <paper_id> <class_label> where class_label is a string.
6. Evaluate your approach in terms of accuracy indicating the percentage of nodes that were predicted correctly.

1. Create a GitHub repository, grant access to the GitHub user heindorf and send an email with the name of the repository to heindorf@uni-paderborn.de.
2. Upload your code (Python preferred) to the repository.
3. Upload your predictions (TSV file) to the repository.
4. Document your approach in the README file. \
Describe what dependencies are required and how to run your code-ideally with a single command. \
Moreover, briefly explain the core idea of your approach in the README file.